<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Dependencies" data-toc-modified-id="Dependencies-1">Dependencies</a></span></li><li><span><a href="#Functions" data-toc-modified-id="Functions-2">Functions</a></span></li><li><span><a href="#Paths" data-toc-modified-id="Paths-3">Paths</a></span></li><li><span><a href="#Main" data-toc-modified-id="Main-4">Main</a></span></li></ul></div>

# Dependencies

In [3]:
options(scipen = 999)

In [4]:
library(ggplot2)
library(ggbeeswarm)
library(Hmisc)
library(circlize)
library(ComplexHeatmap)
library(iNEXT)

# Functions 

In [5]:
div_df.fx <- function(master_df, div_dfCBTTC, div_dfTARGET, div_dfDKFZ, div_dfTCGA){
    message("master_df total number of samples: ",
            dim(master_df)[1])
    print(table(master_df$group))

    message("div_df CBTTC: ")
    print(dim(div_dfCBTTC))

# Merge CBTTC with sample_id
    div_dfCBTTC$filename <- as.character(div_dfCBTTC$filename)
# remove the leading "_" from filename
    div_dfCBTTC$aliquot_id <- substr(div_dfCBTTC$filename,2, nchar(div_dfCBTTC$filename))
    div_dfCBTTC$sample_id  <- gsub("_.*", "", div_dfCBTTC$aliquot_id)
    
    div_dfCBTTC$aliquot_id <- NULL
    div_dfCBTTC$filename <- NULL
     
    master_df_div_CBTTC <- merge(master_df, div_dfCBTTC, by = "sample_id")

    message("div_df CBTTC post-merge: ") 
    print(dim(master_df_div_CBTTC))
    print(table(master_df_div_CBTTC$cohort))
    
# Merge all other cohorts by sample_id
    Div_df <- rbind(div_dfTARGET,div_dfDKFZ,div_dfTCGA)
    message("div_df TCGA, TARGET, DKFZ: ")
    print(dim(Div_df))
    
## there is a leading "_", remove it!    
    Div_df$filename <- as.character(Div_df$filename)
    Div_df$sample_id <- substr(Div_df$filename,2, nchar(Div_df$filename))
## some TCGA ids have leading "RA_", remove it!
    Div_df$sample_id <- gsub("RA_", "", Div_df$sample_id)   
    Div_df$filename <- NULL    
  
    master_df_div <- merge(master_df, Div_df, by = "sample_id")
    message("div_df TCGA,TARGET,DKFZ post-merge: ")
    print(dim(master_df_div))    
    print(table(master_df_div$group))
    
    master_df_div <- rbind(master_df_div,master_df_div_CBTTC)
    
    message("Total div_df post-merge: ")
    print(dim(master_df_div))
    print(table(master_df_div$group))
    return(master_df_div)
}


# Paths

In [6]:
manifestpath <- "/Users/anabbi/OneDrive - UHN/Documents/IPD2/Manifests/"
datapath <- "/Users/anabbi/OneDrive - UHN/Documents/IPD2/Data/"
plotpath <- "/Users/anabbi/OneDrive - UHN/Documents/IPD2/Plots/"

# Main

In [7]:
load(file = file.path(datapath,"TME_clusters/IPD_estimate_metadata_IC.RData"))

In [8]:
table(metadata_IC$group)


 CBTTC   DKFZ TARGET 
   579    195    149 

In [9]:
load(paste0(datapath, "diversity/DKFZ_TRAlist_Div_df.RData"))
load(paste0(datapath, "diversity/TARGET_TRAlist_Div_df.RData"))
load(paste0(datapath, "diversity/CBTTC_TRAlist_Div_df.RData"))
load(paste0(datapath, "diversity/TCGA_Div_df_TRA.RData"))

In [10]:
load(paste0(datapath, "diversity/DKFZ_TRBlist_Div_df.RData"))
load(paste0(datapath, "diversity/TARGET_TRBlist_Div_df.RData"))
load(paste0(datapath, "diversity/CBTTC_TRBlist_Div_df.RData"))
load(paste0(datapath, "diversity/TCGA_Div_df_TRB.RData"))

In [11]:
load(paste0(datapath, "diversity/DKFZ_IGHlist_Div_df.RData"))
load(paste0(datapath, "diversity/TARGET_IGHlist_Div_df.RData"))
load(paste0(datapath, "diversity/CBTTC_IGHlist_Div_df.RData"))
load(paste0(datapath, "diversity/TCGA_Div_df_IGH.RData"))

In [12]:
head(DKFZ_TRBlist_Div_df)

,filename,TRB,Reads,Average_reads,Max_reads,IDis,Singletons,Doubletons,qD,estimated_Richness,observed_Richness,estimated_Shannon,observed_Shannon,estimated_Simpson,observed_Simpson,Divscore
,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,_H049-GWNW,46,58,1.260870,6,0.6981470,40,4,82.135,242.552,46,172.976,39.800,66.120,31.148,28.829333
2,_H049-JVCT,13,16,1.230769,2,0.3563048,10,3,20.283,28.625,13,31.388,12.338,40.000,11.636,15.694000
3,_H049-SPP6,339,712,2.100295,47,2.3167365,242,62,528.246,810.627,339,278.625,156.454,57.618,53.374,5.928191
4,_H049-UV6K,98,111,1.132653,6,0.5324194,91,4,185.047,1123.800,98,844.716,90.318,244.200,76.528,140.786000
5,_ICGC_ET9,3,4,1.333333,2,0.4330127,2,1,4.025,4.500,3,4.891,2.828,2.667,2.667,2.445500
6,_ICGC_GBM2,3,4,1.333333,2,0.4330127,2,1,4.025,4.500,3,4.891,2.828,2.667,2.667,2.445500


In [13]:
metadata_IC_TRA <- div_df.fx(metadata_IC, CBTTC_TRAlist_Div_df,
                             TARGET_TRAlist_Div_df,DKFZ_TRAlist_Div_df,
                             TCGA_Div_df_TRA)

master_df total number of samples: 923




 CBTTC   DKFZ TARGET 
   579    195    149 


div_df CBTTC: 



[1] 354  16


div_df CBTTC post-merge: 



...

[1] 1662   16


div_df TCGA,TARGET,DKFZ post-merge: 



[1] 126  39

  DKFZ TARGET 
    22    104 


Total div_df post-merge: 



[1] 312  39

 CBTTC   DKFZ TARGET 
   186     22    104 


In [14]:
metadata_IC_TRB <- div_df.fx(metadata_IC, CBTTC_TRBlist_Div_df,
                             TARGET_TRBlist_Div_df, DKFZ_TRBlist_Div_df,
                             TCGA_Div_df_TRB)

master_df total number of samples: 923




 CBTTC   DKFZ TARGET 
   579    195    149 


div_df CBTTC: 



[1] 433  16


div_df CBTTC post-merge: 



...

[1] 1730   16


div_df TCGA,TARGET,DKFZ post-merge: 



[1] 127  39

  DKFZ TARGET 
    16    111 


Total div_df post-merge: 



[1] 361  39

 CBTTC   DKFZ TARGET 
   234     16    111 


In [15]:
metadata_IC_IGH <- div_df.fx(metadata_IC, CBTTC_IGHlist_Div_df,
                             TARGET_IGHlist_Div_df, DKFZ_IGHlist_Div_df,
                             TCGA_Div_df_IGH)

master_df total number of samples: 923




 CBTTC   DKFZ TARGET 
   579    195    149 


div_df CBTTC: 



[1] 805  16


div_df CBTTC post-merge: 



...

[1] 2259   16


div_df TCGA,TARGET,DKFZ post-merge: 



[1] 181  39

  DKFZ TARGET 
    47    134 


Total div_df post-merge: 



[1] 624  39

 CBTTC   DKFZ TARGET 
   443     47    134 


In [16]:
metadata_IC_TRA$Divscore <- NULL
metadata_IC_TRB$Divscore <- NULL
metadata_IC_IGH$Divscore <- NULL

In [17]:
table(metadata_IC_TRA$immune_cluster)


 C1  C2  C3  C4 
 78 118  77  39 

In [18]:
save(metadata_IC_TRA,
         file = paste0(datapath, "diversity/metadata_IC_TRA.RData"))


In [19]:
save(metadata_IC_TRB,
         file = paste0(datapath, "diversity/metadata_IC_TRB.RData"))

In [20]:
save(metadata_IC_IGH,
         file = paste0(datapath, "diversity/metadata_IC_IGH.RData"))